# Latency measurements for warehouse_4

Conclusions:
- These metrics are useless apart from "min latency"
- Latency grows over time, as all data is fed as fast as possible (faster than it is possible to process)
- Min latency can depend on many things, average min latency would make more sense but is not possible

Future measurements:
- Run a warehouse experiment that accounts for the max throughput of each experiment scenario (similar to yolo experiment)

In [6]:
import common_utils
import os
import pandas as pd

# Example usage
root_folder = '../../../data_warehouse/minimized_warehouse_4'
filename = 'worker1.feather'
subfolders = common_utils.find_subfolders_with_file(root_folder, filename)
print(subfolders)
prom_data_paths = {os.path.basename(x): x for x in subfolders}
yolo_data_paths = {key: os.path.join(val, "worker_qos.feather") for key, val in prom_data_paths.items()}


['../../../data_warehouse/minimized_warehouse_4/1735854367_(5.5000)', '../../../data_warehouse/minimized_warehouse_4/1735863576_(1.5000)', '../../../data_warehouse/minimized_warehouse_4/1735851224_(7.5000)', '../../../data_warehouse/minimized_warehouse_4/1735849172_(8.1000)', '../../../data_warehouse/minimized_warehouse_4/1735844045_(30.10000)', '../../../data_warehouse/minimized_warehouse_4/1735860195_(2.5000)', '../../../data_warehouse/minimized_warehouse_4/1735853373_(6.10000)', '../../../data_warehouse/minimized_warehouse_4/1735842931_(30.1000)', '../../../data_warehouse/minimized_warehouse_4/1735845629_(20.10000)', '../../../data_warehouse/minimized_warehouse_4/1735859344_(2.1000)', '../../../data_warehouse/minimized_warehouse_4/1735852779_(6.5000)', '../../../data_warehouse/minimized_warehouse_4/1735852257_(6.1000)', '../../../data_warehouse/minimized_warehouse_4/1735848689_(9.10000)', '../../../data_warehouse/minimized_warehouse_4/1735853852_(5.1000)', '../../../data_warehouse/m

In [7]:
from utils.header_cleaner import *
import difflib
import os


"""
Fetch paths to the data
"""


"""
Get corresponding yolo stats for each model 
"""
response_time = {}
for key in prom_data_paths.keys():
    try:
        yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    except:
        print(f"Failed to read {key}")
        continue
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')  # Convert to datetime (optional)
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5s')
    model_info = common_utils.path_to_workers_and_pcl_size(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.num_vehicles] = resampled_df.agg({'end_to_end_response_time': 'mean'}).reset_index()['end_to_end_response_time'].rename(key)


In [8]:
import plotly.express as px
from matplotlib import pyplot as plt

# Function to compute simple moving average
def moving_average(data, window_size):
    return data.rolling(window=window_size).mean()

# You can adjust the window size for different levels of smoothing
window_size = 3

for resolution in sorted(response_time.keys()):
    queue_df = pd.DataFrame.from_dict(response_time[resolution])
    fig = px.line(queue_df, x=queue_df.index, y=queue_df.columns)
    fig.update_layout(title=f'Worker end-to-end latency (PCL resolution: {resolution})', xaxis_title='Time', yaxis_title='Reponse Time in MS', yaxis_type='log')
    fig.show()

In [9]:
import os
import pandas as pd
from functools import lru_cache
from collections import namedtuple
import difflib
import plotly.express as px

response_time = {}
for key in prom_data_paths.keys():
    try:
        yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    except:
        print(f"Failed to read {key}")
        continue
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5S')
    model_info = common_utils.path_to_workers_and_pcl_size(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.num_vehicles] = resampled_df.agg({'end_to_end_response_time': 'min'}).reset_index()['end_to_end_response_time'].rename(key)

data = []
for resolution, models in response_time.items():
    for model, latency in models.items():
        if not latency.empty:
            min_latency = latency.min()
            data.append((str(resolution), model, min_latency))

# Sort data before creating the DataFrame
# sorted_data = sorted(data, key=lambda x: common_utils._sort_key_size_version(x[1]))

df = pd.DataFrame(data, columns=['Resolution', 'Model', 'Min Latency'])
fig = px.bar(df, x='Model', y='Min Latency', color='Resolution', barmode="group", title=f'Minimum end-to-end latency', log_y=True)
fig.update_layout(yaxis_title='Min Latency (ms)')
fig.show()

In [12]:
import os
import pandas as pd
from functools import lru_cache
from collections import namedtuple
import difflib
import plotly.express as px

response_time = {}
for key in prom_data_paths.keys():
    yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5S')
    print(key)
    model_info = common_utils.path_to_workers_and_pcl_size(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.model] = resampled_df.agg({'end_to_end_response_time': 'min'}).reset_index()['end_to_end_response_time'].rename(key)

data = []

# Sorting resolutions to ensure we are correctly calculating differences from previous ones
sorted_resolutions = sorted(response_time.keys())

for resolution in sorted_resolutions:
    for model, latency in response_time[resolution].items():
        if not latency.empty:
            if sorted_resolutions.index(resolution) > 0:
                previous_resolution = sorted_resolutions[sorted_resolutions.index(resolution) - 1]
                if model in response_time[previous_resolution]:
                    previous_latency = response_time[previous_resolution][model]
                    if not previous_latency.empty:
                        min_latency = latency.min() - previous_latency.min()
                        data.append((str(resolution), model, min_latency))
                else:
                    min_latency = latency.min()
                    data.append((str(resolution), model, min_latency))
            else:
                min_latency = latency.min()
                data.append((str(resolution), model, min_latency))

df = pd.DataFrame(data, columns=['Resolution', 'Model', 'Min Latency'])
fig = px.bar(df, x='Model', y='Min Latency', color='Resolution', barmode="stack", title=f'Minimum end-to-end latency (overlapping bars)', log_y=True)
fig.show()
    
    
"""
Same as above, but here the bar peaks match to actual latency values, instead of cumulative latency over all resolutions
"""

1735854367_(5.5000)


AttributeError: 'WarehouseInfo' object has no attribute 'model'